# `auto_arima`

Pyramid bring R's [`auto.arima`](https://www.rdocumentation.org/packages/forecast/versions/7.3/topics/auto.arima) functionality to Python by wrapping statsmodel [`ARIMA`](https://github.com/statsmodels/statsmodels/blob/master/statsmodels/tsa/arima_model.py) and [`SARIMAX`](https://github.com/statsmodels/statsmodels/blob/master/statsmodels/tsa/statespace/sarimax.py) models into a singular scikit-learn-esque estimator ([`pyramid.arima.ARIMA`](https://github.com/tgsmith61591/pyramid/blob/master/pyramid/arima/arima.py)) and adding several layers of degree and seasonal differencing tests to identify the optimal model parameters.

In [1]:
import numpy as np
import pyramid

print('numpy version: %r' % np.__version__)
print('pyramid version: %r' % pyramid.__version__)

numpy version: '1.11.3'
pyramid version: '0.1'


In [2]:
# this is a dataset from R
wineind = np.array([
    15136, 16733, 20016, 17708, 18019,
    19227, 22893, 23739, 21133, 22591,
    26786, 29740, 15028, 17977, 20008,
    21354, 19498, 22125, 25817, 28779,
    20960, 22254, 27392, 29945, 16933,
    17892, 20533, 23569, 22417, 22084,
    26580, 27454, 24081, 23451, 28991,
    31386, 16896, 20045, 23471, 21747,
    25621, 23859, 25500, 30998, 24475,
    23145, 29701, 34365, 17556, 22077,
    25702, 22214, 26886, 23191, 27831,
    35406, 23195, 25110, 30009, 36242,
    18450, 21845, 26488, 22394, 28057,
    25451, 24872, 33424, 24052, 28449,
    33533, 37351, 19969, 21701, 26249,
    24493, 24603, 26485, 30723, 34569,
    26689, 26157, 32064, 38870, 21337,
    19419, 23166, 28286, 24570, 24001,
    33151, 24878, 26804, 28967, 33311,
    40226, 20504, 23060, 23562, 27562,
    23940, 24584, 34303, 25517, 23494,
    29095, 32903, 34379, 16991, 21109,
    23740, 25552, 21752, 20294, 29009,
    25500, 24166, 26960, 31222, 38641,
    14672, 17543, 25453, 32683, 22449,
    22316, 27595, 25451, 25421, 25288,
    32568, 35110, 16052, 22146, 21198,
    19543, 22084, 23816, 29961, 26773,
    26635, 26972, 30207, 38687, 16974,
    21697, 24179, 23757, 25013, 24019,
    30345, 24488, 25156, 25650, 30923,
    37240, 17466, 19463, 24352, 26805,
    25236, 24735, 29356, 31234, 22724,
    28496, 32857, 37198, 13652, 22784,
    23565, 26323, 23779, 27549, 29660,
    23356]
).astype(np.float64)

## Fitting an ARIMA

You do not need to call `auto_arima` in order to fit a model. If you know the order and seasonality of your data, you can simply fit an ARIMA:

In [ ]:
from pyramid.arima import ARIMA

fit = ARIMA(order=(1, 1, 1), seasonal_order=(0, 1, 1, 12)).fit(y=wineind)

## Seasonality

Furthermore, your data does not have to exhibit seasonality to work with an ARIMA:

In [ ]:
fit = ARIMA(order=(1, 1, 1), seasonal_order=None).fit(y=wineind)

## Finding the optimal model parameters using `auto_arima`:

If you are unsure (as is common) of the best parameters for your model, let `auto_arima` figure it out for you:

In [5]:
from pyramid.arima import auto_arima

fit = auto_arima(wineind, start_p=1, start_q=1, max_p=3, max_q=3, m=12,
                 start_P=0, seasonal=True, n_jobs=-1, d=1, D=1,
                 error_action='ignore',  # don't want to know if an order does not work
                 suppress_warnings=True)  # don't want convergence warnings

fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  176
Model:             SARIMAX(3, 1, 3)x(2, 1, 2, 12)   Log Likelihood               -1526.746
Date:                            Thu, 01 Jun 2017   AIC                           3077.492
Time:                                    13:28:04   BIC                           3115.537
Sample:                                         0   HQIC                          3092.923
                                            - 176                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept   -111.0512   1504.110     -0.074      0.941   -3059.052    2836.949
ar.L1         -0.6769     11.260     -0.060      0.952     -22.745      21.392
ar.L2         -0.1089      3.921     -0.028      0.978      -7.793       7.576
ar.L3         -0.0282      0.384     -0.073      0.941      -0.780       0.724
ma.L1          0.0941     11.254      0.008      0.993     -21.964      22.152
ma.L2         -0.4349      2.764     -0.157      0.875      -5.852       4.983
ma.L3         -0.0666      3.646     -0.018      0.985      -7.213       7.080
ar.S.L12      -0.1941     18.463     -0.011      0.992     -36.381      35.992
ar.S.L24       0.0724      3.690      0.020      0.984      -7.160       7.305
ma.S.L12      -0.1928     18.474     -0.010      0.992     -36.401      36.016
ma.S.L24      -0.2260     10.839     -0.021      0.983     -21.470      21.018
sigma2      7.518e+06      0.174   4.33e+07      0.000    7.52e+06    7.52e+06
===================================================================================
Ljung-Box (Q):                       47.06   Jarque-Bera (JB):                19.34
Prob(Q):                              0.21   Prob(JB):                         0.00
Heteroskedasticity (H):               1.17   Skew:                            -0.56
Prob(H) (two-sided):                  0.57   Kurtosis:                         4.25
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number 1.37e+26. Standard errors may be unstable.
"""

## Predicting future values

After your model is fit, you can forecast future values using the `predict` function, just like in sci-kit learn:

In [6]:
next_25 = fit.predict(n_periods=25)
next_25

array([ 21712.33303748,  26161.38404227,  30280.64516821,  35266.92400057,
        12868.21554429,  19690.18089443,  21388.81625939,  23561.96201726,
        21460.56508144,  23555.59664921,  26944.97580824,  22297.46589831,
        19781.93797871,  23518.36419626,  27748.51007399,  32788.86310966,
        10594.56147419,  16890.11311597,  18741.00888118,  20753.23509606,
        18698.59374577,  20411.12269924,  24011.91367651,  19679.1500194 ,
        16916.36835378])

## Visualizing forecasts

In [8]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [10]:
import pandas as pd

def plot_arima(truth, forecasts, title="Forecasted values", xaxis_label='Time',
               yaxis_label='Value', c1='#A6CEE3', c2='#B2DF8A', **kwargs):
    
    # make truth and forecasts into pandas series
    n_truth = truth.shape[0]
    n_forecasts = forecasts.shape[0]
    
    truth = pd.Series(truth, index=np.arange(truth.shape[0]))
    forecasts = pd.Series(forecasts, index=np.arange(n_truth, n_truth + n_forecasts))
    
    # set up the plot
    p = figure(title=title, plot_height=400, **kwargs)
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = yaxis_label
    
    # add the lines
    p.line(truth.index, truth.values, color=c1, legend='Observed')
    p.line(forecasts.index, forecasts.values, color=c2, legend='Forecasted')
    
    return p
    
# call the plotting func
show(plot_arima(wineind, next_25))